In [ ]:
import os
import numpy as np
import skimage.io
import matplotlib
import matplotlib.pyplot as plt
import cv2
from mrcnn.config import Config
from mrcnn import utils
from mrcnn import visualize 
import mrcnn.model as modellib

In [ ]:
class WeldingConfig(Config):
    """Configuration for training on the toy shapes dataset.
    Derives from the base Config class and overrides values specific
    to the toy shapes dataset.
    """
    # Give the configuration a recognizable name
    NAME = "Welding"
    label_classes = 6
 
    # Train on 1 GPU and 8 images per GPU. We can put multiple images on each
    # GPU because the images are small. Batch size is 8 (GPUs * images/GPU).
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
 
    # Number of classes (including background)
    NUM_CLASSES = 1 + label_classes  # background + 1 shapes
 
    # Use small images for faster training. Set the limits of the small side
    # the large side, and that determines the image shape.
    IMAGE_MIN_DIM = 320
    IMAGE_MAX_DIM = 384
 
    # Use smaller anchors because our image and objects are small
    RPN_ANCHOR_SCALES = (8 * 6, 16 * 6, 32 * 6, 64 * 6, 128 * 6)  # anchor side in pixels
 
    # Reduce training ROIs per image because the images are small and have
    # few objects. Aim to allow ROI sampling to pick 33% positive ROIs.
    TRAIN_ROIS_PER_IMAGE = 100
 
    # Use a small epoch since the data is simple
    STEPS_PER_EPOCH = 100
 
    # use small validation steps since the epoch is small
    VALIDATION_STEPS = 50

In [ ]:
# Directory to save logs and trained model
ROOT_DIR = os.getcwd()
MODEL_DIR = os.path.join(ROOT_DIR, "logs")
MODEL_PATH = os.path.join(MODEL_DIR, "welding20220527T1235\\mask_rcnn_welding_0035.h5")

# Directory of images to run detection on
IMAGE_DIR = os.path.join(ROOT_DIR, "images")
config = WeldingConfig()

# Create model object in inference mode.
model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)
 
# Load weights
model.load_weights(MODEL_PATH, by_name=True)

In [ ]:
# Load a random image from the images folder
Image_DIR = os.path.join(ROOT_DIR, "test_data/")

for i in os.listdir(Image_DIR):
    if i[-3:] != 'png':
        continue
    img_path = os.path.join(Image_DIR, i)
    print(img_path)
    image = skimage.io.imread(img_path)
    # Run detection
    results = model.detect([image], verbose=1)

    # Visualize results
    r=results[0]
    visualize.display_instances(image,r['rois'], r['masks'], 
                                r['class_ids'], class_names, r['scores'])